In [1]:
from sims_pars.fitting.base import AbsObjectiveSimBased
from sims_pars import bayes_net_from_script
import scipy.stats as sts

In [2]:
class TestCase(AbsObjectiveSimBased):
    def __init__(self, mu, n=10):
        bn = bayes_net_from_script('''
        PCore Normal2 {
            mu1 ~ norm(0, 1)
            mu2 ~ norm(0, 1)
            sd ~ gamma(0.1, 0.1)
        }
        ''')
        AbsObjectiveSimBased.__init__(self, bn)
        self.Mu = mu
        self.N = n
        self.X1 = sts.norm(self.Mu[0], 1).rvs(n)
        self.X2 = sts.norm(self.Mu[1], 1).rvs(n)

    def simulate(self, pars):
        
        sts.norm(pars['mu2'], pars['sd']).rvs(self.N)
        
        return {
            'mu1': sts.norm(pars['mu1'], pars['sd']).rvs(10).mean(),
            'mu2': sts.norm(pars['mu2'], pars['sd']).rvs(10).mean()
        }

    def link_likelihood(self, sim):
        return sts.norm.logpdf(self.X1, sim['mu1'], 1).sum() + sts.norm.logpdf(self.X2, sim['mu2'], 1).sum()

model = TestCase([10, -5])
    
    
print('Free parameters: ', model.FreeParameters)



Free parameters:  ['mu1', 'mu2', 'sd']


## Read Model Info

## 

In [5]:
di = model.BayesianNetwork['mu1'].get_distribution()

In [7]:
di.mean(), di.std()

(0.0, 1.0)

In [6]:
from sims_pars.fit.converter import *
import numpy as np

In [7]:
converters = get_converters(model.Domain)
converters

[U(0, 1) <-> [-Inf, Inf], U(0, 1) <-> [-Inf, Inf], U(0, 1) <-> [0.0, Inf]]

In [60]:
from abc import ABCMeta, abstractmethod
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

class AbsEmulator(metaclass=ABCMeta):
    def __init__(self, output, kernel=None, **kwargs):
        self.Output = output
        if kernel is None:
            self.Kernel = RBF(length_scale=1e-10)
        else:
            self.Kernel = kernel
        self.Opt = dict(kwargs)
        self.GP = None

    @abstractmethod
    def train(self, xs, ys) -> None:
        pass

    @abstractmethod
    def predict(self, xs) -> tuple[np.ndarray, np.ndarray]:
        pass
    
    
class GPREmulator(AbsEmulator):
    def train(self, xs, ys):
        ys = np.array([[y[self.Output]] for y in ys])
        self.GP = GaussianProcessRegressor(kernel=self.Kernel, n_restarts_optimizer=5, **self.Opt)
        self.GP.fit(xs, ys)
        
    def predict(self, xs) -> tuple[list, list]:
        assert self.GP is not None
        mean, var = self.GP.predict(xs, return_std=True)
        return mean, var

In [57]:
ps = [model.sample_prior() for _ in range(20)]
sims = [model.simulate(p) for p in ps]

xs = np.zeros((len(ps), len(converters)))
for i, p in enumerate(ps):
    for j, dom in enumerate(model.Domain):
        xs[i, j] = converters[j].value2uniform(p[dom.Name])

xs

array([[8.59325208e-01, 2.45381284e-01, 5.07657201e-06],
       [6.87061547e-01, 2.70048766e-01, 6.59347210e-03],
       [2.29876311e-01, 5.49500581e-01, 2.22044605e-16],
       [3.52363214e-01, 1.67939865e-01, 1.93275396e-11],
       [9.42699773e-01, 1.33084658e-01, 2.10143726e-07],
       [5.00669452e-01, 5.46614180e-01, 9.32698363e-13],
       [8.51602031e-01, 4.16336045e-01, 2.19224638e-12],
       [7.04423987e-01, 3.78568965e-01, 2.60415390e-03],
       [1.66428883e-01, 1.56702233e-01, 5.99195459e-01],
       [1.48403303e-01, 6.66545188e-01, 2.12448119e-03],
       [6.39055907e-01, 7.58766422e-01, 9.91624700e-01],
       [7.19741396e-01, 2.13470495e-01, 4.07717759e-01],
       [7.73475493e-01, 4.60487274e-01, 2.18895068e-03],
       [1.68723770e-01, 4.69105864e-01, 1.95979026e-08],
       [4.74332299e-01, 4.06794237e-01, 4.98880287e-02],
       [1.74228680e-01, 5.46249266e-01, 7.75396423e-01],
       [1.09298848e-01, 7.74939776e-01, 9.79123198e-03],
       [8.78502104e-01, 5.96738

In [61]:
emu = GPREmulator('mu1')

In [62]:
emu.train(xs, sims)

In [63]:
e, v = emu.predict(xs)

In [64]:
s = [sim['mu1'] for sim in sims]

In [65]:
(e - s) / s

array([ 1.98071247e-09, -1.00990947e-10,  2.36296041e-10, -3.71525684e-10,
       -6.22752788e-10, -1.49498440e-07, -1.79194943e-09, -4.52863960e-09,
       -6.96104139e-11,  8.06477816e-10, -1.10450712e-10, -1.45750554e-10,
        3.99257096e-09, -3.26235129e-09, -8.99408359e-09, -9.91841297e-11,
       -4.65422558e-10, -3.02043404e-10,  2.89503672e-09,  3.23879298e-10])